### 1.Wiki Web Scraping for Toronto Postal

In [160]:
# Web Scraping Tutorial: https://www.youtube.com/watch?v=ng2o98k983k

In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from geopy.geocoders import Nominatim
import folium
# import geocoder

In [2]:
Toronto_Postal=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(Toronto_Postal, 'lxml')

In [3]:
bodytext = soup.find('table', class_= "wikitable") #locate the tabledata tag
table = bodytext.find_all('td') # return a list with all 'td'

In [4]:
#Scraping each column text of table into List and final assemble DataFrame
Postalcode = []
Borough = []
Neighborhood = []
for i in range(len(table)):
    if i%3 ==0:
        Postalcode.append(table[i].get_text().split('\n')[0])
    if i%3 ==1:
        Borough.append(table[i].get_text().split('\n')[0])
    if i%3 ==2:
        Neighborhood.append(table[i].get_text().split('\n')[0])
# print(Postalcode, Borough, Neighborhood)
df_Torontopo = pd.DataFrame(list(zip(Postalcode, Borough, Neighborhood )), columns = ['Postal Code', 'Borough','Neighborhood'])

In [5]:
df_Torontopo_clean = df_Torontopo[~(df_Torontopo['Borough'] == 'Not assigned')]
df_Torontopo_clean.Neighborhood = df_Torontopo_clean['Neighborhood'].str.replace('/',',')
df_Torontopo_clean.head()


C:\Users\PC\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [6]:
print(df_Torontopo_clean.shape)

(103, 3)


### 2. Get Latitude and Longtitude

In [7]:
# import provided Geo Postocal CSV file
filename = r'C:\Users\PC\Desktop\Coursara Jupyter Notebook\Capstone\Geospatial_Coordinates.csv'
df_geo = pd.read_csv(filename)
df_geo.head()
df_geo['Postal Code'][0]

'M1B'

In [93]:
df_T = pd.merge(df_Torontopo_clean, df_geo, on = 'Postal Code', how = 'left' )
df_T.sort_values('Borough').head(50)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
86,M4V,Central Toronto,"Summerhill West , Rathnelly , South Hill , For...",43.686412,-79.400049
79,M4S,Central Toronto,Davisville,43.704324,-79.388790
83,M4T,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160
68,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
74,M5R,Central Toronto,"The Annex , North Midtown , Yorkville",43.672710,-79.405678
67,M4P,Central Toronto,Davisville North,43.712751,-79.390197
62,M5N,Central Toronto,Roselawn,43.711695,-79.416936
73,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


### 3 Explore and Cluster Neighborhoods

The observation is that the second digit of Postal Code used number encodings, and some of the same boroughs have different numbers. The below clustering is trying build a relation connecting the Postal Code with Borough.

-Use different Circle Marks color to separate the distinct encodings within the Postal Code. For Example, "M4J" is designated for Borough 'East York' and 'M4T' for	'Central Toronto', but they share the same encoding digit of '4'. 


In [94]:
map_toronto = folium.Map(location = [43.712751 , -79.390197], zoom_start = 11)
colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'white', 'black', 'cadetblue']
Borough_list = list(df_T.Borough.unique())
for lat, lng, borough, Post, Neighbor in zip(df_T['Latitude'], df_T['Longitude'], df_T['Borough'],df_T['Postal Code'],df_T['Neighborhood']):
    label = '{}, {}, {}'.format(Post, Neighbor, borough)
    label = folium.Popup(label, parse_html=True)
    Po_color = colors[int(list(Post)[1])-1]
    folium.CircleMarker(
        [lat, lng],
        radius = 8,
        popup=label,
        color=Po_color,
        fill=True,
        fill_color='lightgray',
        fill_opacity=0.9,
        parse_html=False).add_to(map_toronto)  
map_toronto